In [10]:
from music21 import *
import glob
import numpy as np
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.callbacks import History, ModelCheckpoint
from keras.optimizers import RMSprop

In [11]:
path = '/home/eurismar/notebooks/Music_Generator_Demo/Pop_Music_Midi/'
#path = '/home/eurismar/notebooks/c-rnn-gan/datadir/classical/chopin'

In [12]:
files = glob.glob('{}/*.mid*'.format(path))

In [13]:
melody_corpus = []
#for file in files[:1]:
for file in files:
    midi_stream = converter.parse(file)
    part0 = midi_stream[0]
    key_sig = part0.keySignature
    for m in part0.measures(1, None):
        for nr in m.flat.notesAndRests:
            pitch = nr.pitch.midi  if isinstance(nr, note.Note) else 0
            duration = float(nr.quarterLength)
            note_repr = (pitch, duration)
            melody_corpus.append(note_repr)

In [14]:
melody_corpus

[(0, 1.0),
 (60, 0.25),
 (60, 0.25),
 (0, 0.5),
 (60, 0.25),
 (60, 0.25),
 (0, 0.5),
 (60, 0.25),
 (60, 0.25),
 (0, 0.5),
 (60, 1.0),
 (64, 0.25),
 (64, 0.5),
 (64, 0.5),
 (62, 0.5),
 (60, 0.25),
 (62, 0.25),
 (60, 0.25),
 (62, 0.5),
 (57, 0.5),
 (0, 0.5),
 (60, 0.25),
 (60, 0.25),
 (0, 0.5),
 (60, 0.25),
 (60, 0.25),
 (0, 0.5),
 (60, 0.25),
 (60, 0.25),
 (0, 0.5),
 (60, 1.0),
 (64, 0.25),
 (64, 0.5),
 (64, 0.75),
 (64, 0.25),
 (62, 0.5),
 (60, 0.5),
 (0, 0.25),
 (53, 0.5),
 (60, 0.5),
 (60, 0.5),
 (0, 0.5),
 (53, 0.25),
 (64, 0.25),
 (64, 0.25),
 (64, 1.0),
 (0, 0.25),
 (53, 0.5),
 (60, 0.5),
 (60, 0.5),
 (0, 0.25),
 (53, 0.25),
 (53, 0.25),
 (64, 0.25),
 (64, 0.25),
 (64, 1.0),
 (0, 0.25),
 (55, 0.5),
 (60, 0.5),
 (60, 0.5),
 (0, 0.5),
 (55, 0.25),
 (60, 0.25),
 (64, 1.0),
 (65, 0.25),
 (64, 0.25),
 (62, 2.0),
 (0, 0.5),
 (62, 0.5),
 (65, 1.0),
 (64, 0.5),
 (64, 1.5),
 (0, 1.0),
 (0, 1.0),
 (57, 0.5),
 (57, 0.5),
 (57, 0.5),
 (60, 0.5),
 (57, 0.5),
 (65, 0.5),
 (65, 1.0),
 (64, 0.5),

In [15]:
melody_set = set(melody_corpus)

In [16]:
melody_set

{(0, 0.25),
 (0, 0.5),
 (0, 0.75),
 (0, 1.0),
 (0, 1.25),
 (0, 1.5),
 (0, 1.75),
 (0, 2.0),
 (0, 2.25),
 (0, 2.5),
 (0, 2.75),
 (0, 3.0),
 (0, 3.25),
 (0, 3.5),
 (0, 4.0),
 (43, 0.25),
 (43, 0.5),
 (43, 0.75),
 (43, 1.5),
 (45, 0.25),
 (45, 0.5),
 (45, 0.75),
 (45, 1.5),
 (45, 2.0),
 (47, 1.0),
 (48, 0.25),
 (48, 0.5),
 (48, 0.75),
 (48, 1.0),
 (48, 1.25),
 (48, 1.5),
 (48, 1.75),
 (48, 2.0),
 (50, 0.25),
 (50, 0.5),
 (50, 0.75),
 (50, 1.0),
 (52, 0.25),
 (52, 0.5),
 (52, 0.75),
 (52, 1.0),
 (52, 1.25),
 (52, 1.5),
 (52, 2.0),
 (53, 0.25),
 (53, 0.5),
 (53, 0.75),
 (53, 1.0),
 (55, 0.25),
 (55, 0.5),
 (55, 0.75),
 (55, 1.0),
 (55, 1.25),
 (55, 1.5),
 (55, 2.0),
 (55, 3.5),
 (55, 4.0),
 (56, 0.5),
 (56, 1.0),
 (57, 0.25),
 (57, 0.5),
 (57, 0.75),
 (57, 1.0),
 (57, 1.25),
 (57, 1.5),
 (57, 1.75),
 (57, 2.0),
 (57, 3.0),
 (59, 0.25),
 (59, 0.5),
 (59, 0.75),
 (59, 1.0),
 (59, 1.25),
 (59, 1.5),
 (59, 2.0),
 (59, 3.0),
 (60, 0.25),
 (60, 0.5),
 (60, 0.75),
 (60, 1.0),
 (60, 1.25),
 (60, 1.

In [17]:
len(melody_corpus)

6999

In [18]:
len(melody_set)

231

In [19]:
notes_indices = {note: i for i, note in enumerate(melody_set)}
indices_notes = {i: note for i, note in enumerate(melody_set)}

In [20]:
notes_indices

{(0, 0.25): 204,
 (0, 0.5): 203,
 (0, 0.75): 205,
 (0, 1.0): 38,
 (0, 1.25): 41,
 (0, 1.5): 39,
 (0, 1.75): 40,
 (0, 2.0): 106,
 (0, 2.25): 108,
 (0, 2.5): 105,
 (0, 2.75): 107,
 (0, 3.0): 177,
 (0, 3.25): 179,
 (0, 3.5): 178,
 (0, 4.0): 21,
 (43, 0.25): 159,
 (43, 0.5): 161,
 (43, 0.75): 160,
 (43, 1.5): 81,
 (45, 0.25): 185,
 (45, 0.5): 180,
 (45, 0.75): 188,
 (45, 1.5): 111,
 (45, 2.0): 42,
 (47, 1.0): 135,
 (48, 0.25): 162,
 (48, 0.5): 164,
 (48, 0.75): 163,
 (48, 1.0): 10,
 (48, 1.25): 9,
 (48, 1.5): 11,
 (48, 1.75): 12,
 (48, 2.0): 74,
 (50, 0.25): 183,
 (50, 0.5): 184,
 (50, 0.75): 79,
 (50, 1.0): 24,
 (52, 0.25): 206,
 (52, 0.5): 207,
 (52, 0.75): 208,
 (52, 1.0): 43,
 (52, 1.25): 45,
 (52, 1.5): 44,
 (52, 2.0): 114,
 (53, 0.25): 93,
 (53, 0.5): 92,
 (53, 0.75): 94,
 (53, 1.0): 29,
 (55, 0.25): 121,
 (55, 0.5): 120,
 (55, 0.75): 122,
 (55, 1.0): 46,
 (55, 1.25): 47,
 (55, 1.5): 49,
 (55, 2.0): 30,
 (55, 3.5): 196,
 (55, 4.0): 69,
 (56, 0.5): 75,
 (56, 1.0): 155,
 (57, 0.25): 21

In [21]:
indices_notes

{0: (59, 0.5),
 1: (59, 0.25),
 2: (59, 0.75),
 3: (64, 0.25),
 4: (64, 0.5),
 5: (64, 0.75),
 6: (62, 2.0),
 7: (62, 2.5),
 8: (62, 2.75),
 9: (48, 1.25),
 10: (48, 1.0),
 11: (48, 1.5),
 12: (48, 1.75),
 13: (74, 1.5),
 14: (74, 1.0),
 15: (77, 1.5),
 16: (77, 1.0),
 17: (79, 2.0),
 18: (79, 2.5),
 19: (84, 2.5),
 20: (66, 0.5),
 21: (0, 4.0),
 22: (69, 2.0),
 23: (69, 2.5),
 24: (50, 1.0),
 25: (74, 2.0),
 26: (74, 2.5),
 27: (76, 1.5),
 28: (76, 1.0),
 29: (53, 1.0),
 30: (55, 2.0),
 31: (79, 1.5),
 32: (79, 1.0),
 33: (79, 1.25),
 34: (60, 2.0),
 35: (60, 2.5),
 36: (63, 0.5),
 37: (68, 0.5),
 38: (0, 1.0),
 39: (0, 1.5),
 40: (0, 1.75),
 41: (0, 1.25),
 42: (45, 2.0),
 43: (52, 1.0),
 44: (52, 1.5),
 45: (52, 1.25),
 46: (55, 1.0),
 47: (55, 1.25),
 48: (64, 4.0),
 49: (55, 1.5),
 50: (88, 1.0),
 51: (88, 1.5),
 52: (65, 1.0),
 53: (65, 1.5),
 54: (86, 3.0),
 55: (67, 2.5),
 56: (67, 2.0),
 57: (72, 2.5),
 58: (72, 2.0),
 59: (75, 0.5),
 60: (80, 0.5),
 61: (80, 0.75),
 62: (57, 

In [22]:
corpus_size = len(melody_set)

In [23]:
corpus_size

231

In [24]:
phrase_len = 20
step_size = 3

In [25]:
phrases = []
next_notes = []

In [26]:
for i in range(0, len(melody_corpus) - phrase_len, step_size):
    phrases.append(melody_corpus[i: i + phrase_len])
    next_notes.append(melody_corpus[i + phrase_len])
print('nb sequences:', len(phrases))

nb sequences: 2327


In [27]:
# transform data into binary matrices
X = np.zeros((len(phrases), phrase_len, corpus_size), dtype=np.bool)
y = np.zeros((len(phrases), corpus_size), dtype=np.bool)
#X = np.zeros((len(phrases), phrase_len, corpus_size))
#y = np.zeros((len(phrases), corpus_size))

In [28]:
X.shape

(2327, 20, 231)

In [29]:
y.shape

(2327, 231)

In [30]:
for i, phrase in enumerate(phrases):
    for j, note in enumerate(phrase):
        X[i, j, notes_indices[note]] = 1
    y[i, notes_indices[next_notes[i]]] = 1

In [31]:
X[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [32]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [33]:
layer_size=128
batch_size = 128
nb_epoch = 2

In [34]:
model = Sequential()
model.add(LSTM(layer_size, return_sequences=True, input_shape=(phrase_len, corpus_size)))
model.add(Dropout(0.2))
model.add(LSTM(layer_size, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(corpus_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop())

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 128)           184320    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 231)               29799     
_________________________________________________________________
activation_1 (Activation)    (None, 231)               0         
Total params: 345,703
Trainable params: 345,703
Non-trainable params: 0
_________________________________________________________________


In [36]:
history = History()

In [37]:
model.fit(X, y, batch_size=batch_size, epochs=nb_epoch, callbacks=[history])

Epoch 1/2
2327/2327 [==============================] - 2s - loss: 4.6580     
Epoch 2/2
2327/2327 [==============================] - 1s - loss: 4.1867     


In [38]:
gen_melody_indices = np.zeros((1, phrase_len, len(melody_set)))

In [39]:
gen_melody_indices

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [40]:
gen_melody_indices.shape


(1, 20, 231)

In [41]:
start_pos = np.random.randint(0, len(melody_corpus) - phrase_len)

In [42]:
seed_phrase = melody_corpus[start_pos : start_pos + phrase_len]

In [43]:
start_pos

3270

In [44]:
seed_phrase

[(76, 1.0),
 (76, 1.0),
 (76, 1.0),
 (76, 1.0),
 (76, 1.0),
 (77, 1.0),
 (77, 2.0),
 (77, 2.0),
 (84, 1.0),
 (83, 0.5),
 (84, 1.0),
 (83, 1.0),
 (81, 0.5),
 (76, 0.25),
 (79, 0.25),
 (76, 0.5),
 (74, 0.5),
 (0, 0.5),
 (74, 0.25),
 (74, 0.25)]

In [45]:
gen_melody = seed_phrase

In [46]:
gen_melody

[(76, 1.0),
 (76, 1.0),
 (76, 1.0),
 (76, 1.0),
 (76, 1.0),
 (77, 1.0),
 (77, 2.0),
 (77, 2.0),
 (84, 1.0),
 (83, 0.5),
 (84, 1.0),
 (83, 1.0),
 (81, 0.5),
 (76, 0.25),
 (79, 0.25),
 (76, 0.5),
 (74, 0.5),
 (0, 0.5),
 (74, 0.25),
 (74, 0.25)]

In [78]:
seq_len = 20

In [79]:
for _ in range(seq_len):
    seed_phrase = gen_melody[-phrase_len:]
    #print('----')
    #print(seed_phrase)
    for i, note in enumerate(seed_phrase):
        #print(i,note)
        gen_melody_indices[0, i, notes_indices[note]] = 1
    x = gen_melody_indices
    #next_note = __predict(model, x, indices_notes, temperature=2)
    preds = model.predict(x, verbose=0)[0]
    preds = np.asarray(preds).astype('float64')
    temperature = 1.0
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    next_index =  np.argmax(probas)
    next_val = indices_notes[next_index]
    print(next_val)

(0, 0.5)
(69, 3.5)
(0, 1.0)
(60, 0.5)
(76, 0.5)
(75, 1.0)
(76, 0.5)
(72, 0.25)
(67, 0.5)
(83, 2.5)
(64, 0.5)
(72, 1.0)
(55, 1.25)
(64, 0.5)
(71, 1.0)
(77, 0.75)
(76, 0.5)
(60, 1.5)
(67, 0.5)
(0, 0.75)
